In [14]:
import glob
fs_cmb=sorted(glob.glob('/Users/mgrecu/GPM/ORO/2A-CS/2B*CONUS*HDF5'))
fs_1GMI=sorted(glob.glob('/Users/mgrecu/GPM/ORO/2A-CS/1C*CONUS*HDF5'))
print(len(fs_cmb))
print(len(fs_1GMI))
#stop
import netCDF4 as nc
from pyresample import kd_tree, geometry
from pyresample.kd_tree import resample_nearest, resample_gauss, resample_custom
def readCMB(fname): # reads relevant data from the CMB file
    fh_cmb=nc.Dataset(fname)
    qv=fh_cmb["KuKaGMI/vaporDensity"][:,:,:]
    press=fh_cmb["KuKaGMI/airPressure"][:,:,:]
    envNodes=fh_cmb["KuKaGMI/envParamNode"][:,:,:]
    airTemp=fh_cmb["KuKaGMI/airTemperature"][:,:,:]
    skTemp=fh_cmb["KuKaGMI/skinTemperature"][:,:]
    binNodes=fh_cmb["KuKaGMI/phaseBinNodes"][:,:]
    pwc=fh_cmb["KuKaGMI/precipTotWaterCont"][:,:,:]
    sfcEmiss=fh_cmb["KuKaGMI/surfEmissivity"][:,:,:]
    dm=fh_cmb["KuKaGMI/precipTotDm"][:,:,:]
    cldw=fh_cmb["KuKaGMI/cloudLiqWaterCont"][:,:,:]
    sfcBin=fh_cmb["KuKaGMI/Input/surfaceRangeBin"][:,:,:]
    zCorrected=fh_cmb["KuKaGMI/correctedReflectFactor"][:,:,:]
    pType=fh_cmb["KuKaGMI/Input/precipitationType"][:,:]
    lon=fh_cmb["KuKaGMI/Longitude"][:,:]
    lat=fh_cmb["KuKaGMI/Latitude"][:,:]
    return qv,press,envNodes,airTemp,skTemp,binNodes,pwc,sfcEmiss,dm,cldw,sfcBin,zCorrected,pType,lon,lat

def readCMB_ray(fname,j): # reads relevant data from the CMB file
    fh_cmb=nc.Dataset(fname)
    qv=fh_cmb["KuKaGMI/vaporDensity"][:,j,:]
    press=fh_cmb["KuKaGMI/airPressure"][:,j,:]
    envNodes=fh_cmb["KuKaGMI/envParamNode"][:,j,:]
    airTemp=fh_cmb["KuKaGMI/airTemperature"][:,j,:]
    skTemp=fh_cmb["KuKaGMI/skinTemperature"][:,j]
    binNodes=fh_cmb["KuKaGMI/phaseBinNodes"][:,j]
    pwc=fh_cmb["KuKaGMI/precipTotWaterCont"][:,j,:]
    sfcEmiss=fh_cmb["KuKaGMI/surfEmissivity"][:,j,:]
    dm=fh_cmb["KuKaGMI/precipTotDm"][:,j,:]
    cldw=fh_cmb["KuKaGMI/cloudLiqWaterCont"][:,j,:]
    sfcBin=fh_cmb["KuKaGMI/Input/surfaceRangeBin"][:,j,:]
    zCorrected=fh_cmb["KuKaGMI/correctedReflectFactor"][:,j,:]
    pType=fh_cmb["KuKaGMI/Input/precipitationType"][:,j]
    lon=fh_cmb["KuKaGMI/Longitude"][:,j]
    lat=fh_cmb["KuKaGMI/Latitude"][:,j]
    #print(fh_cmb["KuKaGMI/Input/"])
    surfaceType=fh_cmb["KuKaGMI/Input/surfaceType"][:,j]
    return qv,press,envNodes,airTemp,skTemp,binNodes,pwc,sfcEmiss,dm,cldw,sfcBin,zCorrected,pType,lon,lat,surfaceType

def read1CGMI(fname): # reads relevant data from the 1C GMI file
    with nc.Dataset(fname) as f:
        #print(f)
        #print(f['S1'])
        lat_s1=f['S1/Latitude'][:]
        lon_s1=f['S1/Longitude'][:]
        tc_s1=f['S1/Tc'][:]
        lat_s2=f['S2/Latitude'][:]
        lon_s2=f['S2/Longitude'][:]
        tc_s2=f['S2/Tc'][:]
    return lat_s1,lon_s1,tc_s1,lat_s2,lon_s2,tc_s2

659
734


In [41]:
imatched=0
for f1 in fs_cmb:
    orb=f1.split('.')[-3]
    for f2 in fs_1GMI:
        if orb in f2:
            imatched+=1
    print(f1,f2)

print(imatched)

/Users/mgrecu/GPM/ORO/2A-CS/2B-CS-CONUS.GPM.DPRGMI.CORRA2022.20190501-S011631-E012509.029377.V07A.HDF5 /Users/mgrecu/GPM/ORO/2A-CS/1C-R-CS-CONUS.GPM.GMI.XCAL2016-C.20190730-S225421-E230438.030791.V07A.HDF5
/Users/mgrecu/GPM/ORO/2A-CS/2B-CS-CONUS.GPM.DPRGMI.CORRA2022.20190501-S025200-E025742.029378.V07A.HDF5 /Users/mgrecu/GPM/ORO/2A-CS/1C-R-CS-CONUS.GPM.GMI.XCAL2016-C.20190730-S225421-E230438.030791.V07A.HDF5
/Users/mgrecu/GPM/ORO/2A-CS/2B-CS-CONUS.GPM.DPRGMI.CORRA2022.20190501-S075135-E075444.029381.V07A.HDF5 /Users/mgrecu/GPM/ORO/2A-CS/1C-R-CS-CONUS.GPM.GMI.XCAL2016-C.20190730-S225421-E230438.030791.V07A.HDF5
/Users/mgrecu/GPM/ORO/2A-CS/2B-CS-CONUS.GPM.DPRGMI.CORRA2022.20190501-S092409-E093249.029382.V07A.HDF5 /Users/mgrecu/GPM/ORO/2A-CS/1C-R-CS-CONUS.GPM.GMI.XCAL2016-C.20190730-S225421-E230438.030791.V07A.HDF5
/Users/mgrecu/GPM/ORO/2A-CS/2B-CS-CONUS.GPM.DPRGMI.CORRA2022.20190501-S105642-E110522.029383.V07A.HDF5 /Users/mgrecu/GPM/ORO/2A-CS/1C-R-CS-CONUS.GPM.GMI.XCAL2016-C.20190730-S22

In [19]:
ntot=0
import numpy as np
j=24
ien=0
import matplotlib.pyplot as plt
import xarray as xr
for f in fs_cmb[:]:
    qv,press,envNodes,airTemp,skTemp,binNodes,pwc,sfcEmiss,dm,cldw,sfcBin,zCorrected,pType,lon,lat,sfcType=readCMB_ray(f,j)
    orb=f.split('.')[-3]
    imatched=0
    for f2 in fs_1GMI:
        if orb in f2:
            imatched=1
            lat_s1,lon_s1,tc_s1,lat_s2,lon_s2,tc_s2=read1CGMI(f2)
            input_def = geometry.SwathDefinition(lons=lon_s1[:,:], lats=lat_s1[:,:])
            output_def = geometry.SwathDefinition(lons=lon[:,np.newaxis], lats=lat[:,np.newaxis])
# Resample the tb_s1 data to the CMB grid using gaussian resampling
            wf = lambda r: 1 - r/20000.0
            tb_s1_resampled = resample_custom(input_def, tc_s1[:,:,:], output_def, radius_of_influence=30000, neighbours=10, weight_funcs=[wf for k in range(9)], fill_value=None)
            input_def = geometry.SwathDefinition(lons=lon_s2[:,:], lats=lat_s2[:,:])
            tb_s2_resampled = resample_custom(input_def, tc_s2[:,:,:], output_def, radius_of_influence=30000, neighbours=10, weight_funcs=[wf for k in range(4)], fill_value=None)
            break
    #break
    if imatched==0:
        continue
    pType=(pType/1e7).astype(int)
    #break
    a=np.nonzero(pType[:]>0)
    if len(a[0])<10:
        continue
    plt.pcolormesh(zCorrected[a[0],:,0].T,vmin=0,vmax=50,cmap='jet')
    plt.ylim(87,40)
    plt.savefig('Images/fig'+str(ien)+'.png')
    plt.close('all')
    ien+=1
    ntot+=len(a[0])
    tc_a=xr.DataArray(np.concatenate((tb_s1_resampled[:,0,:],tb_s2_resampled[:,0,:]),axis=1),dims=['all_scan','n13'],coords={'all_scan':range(pType.shape[0]),'n13':np.arange(13)})
    qv_a=xr.DataArray(qv[a[0],:],dims=['scan','n9'],coords={'scan':a[0],'n9':np.arange(10)})
    press_a=xr.DataArray(press[a[0],:],dims=['scan','n9'],coords={'scan':a[0],'n9':np.arange(10)})
    envNodes_a=xr.DataArray(envNodes[a[0],:],dims=['scan','n9'],coords={'scan':a[0],'n9':np.arange(10)})
    airTemp_a=xr.DataArray(airTemp[a[0],:],dims=['scan','n9'],coords={'scan':a[0],'n9':np.arange(10)})
    skTemp_a=xr.DataArray(skTemp[a[0]],dims=['scan'],coords={'scan':a[0]})
    binNodes_a=xr.DataArray(binNodes[a[0]],dims=['scan','n5'],coords={'scan':a[0],'n5':np.arange(5)})
    pwc_a=xr.DataArray(pwc[a[0],:],dims=['scan','nbins'],coords={'scan':a[0],'nbins':np.arange(88)})
    sfcEmiss_a=xr.DataArray(sfcEmiss[a[0],:],dims=['scan','nchan'],coords={'scan':a[0],'nchan':np.arange(13)})
    dm_a=xr.DataArray(dm[a[0],:],dims=['scan','nbins'],coords={'scan':a[0],'nbins':np.arange(88)})
    cldw_a=xr.DataArray(cldw[a[0],:],dims=['scan','nbins'],coords={'scan':a[0],'nbins':np.arange(88)})
    sfcBin_a=xr.DataArray(sfcBin[a[0],0],dims=['scan'],coords={'scan':a[0]})
    zCorrected_a=xr.DataArray(zCorrected[a[0],:,:],dims=['scan','nbins','nf'],coords={'scan':a[0],'nbins':np.arange(88),'nf':np.arange(2)})
    pType_a=xr.DataArray(pType[a[0]],dims=['scan'],coords={'scan':a[0]})
    lon_a=xr.DataArray(lon[a[0]],dims=['scan'],coords={'scan':a[0]})
    lat_a=xr.DataArray(lat[a[0]],dims=['scan'],coords={'scan':a[0]})
    sfcType_a=xr.DataArray(sfcType[a[0]],dims=['scan'],coords={'scan':a[0]})
    ds=xr.Dataset({'qv':qv_a,'press':press_a,'envNodes':envNodes_a,'airTemp':airTemp_a,'skTemp':skTemp_a,'binNodes':binNodes_a,'pwc':pwc_a,'sfcEmiss':sfcEmiss_a,'dm':dm_a,'cldw':cldw_a,'sfcBin':sfcBin_a,'zCorrected':zCorrected_a,'pType':pType_a,'lon':lon_a,'lat':lat_a,'sfcType':sfcType_a,'tc':tc_a})
    compLev=5
    encoding = {var: {'zlib': True, 'complevel': compLev} for var in ds.data_vars}
    ds.to_netcdf('subsets/CORRA_CONUS_subset%3.3i.nc'%ien,encoding=encoding)
    #break
    

In [16]:
print(zCorrected.shape)

(741, 88, 2)


In [6]:
print(tc_s2.shape)

(329, 221, 4)


In [37]:
print(f.split('.')[-3])    

032082
